In [64]:
from func_aux import *

Teste com pesos normais

In [65]:
def calculate_score_difference(estado, jogador):
    player_score = estado.state[estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.NORTH_SCORE_PIT]
    opponent_score = estado.state[estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT]
    return player_score - opponent_score

def calculate_seeds_difference(estado, jogador):
    player_pits = estado.state[:estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT]
    opponent_pits = estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[:estado.SOUTH_SCORE_PIT]
    return sum(player_pits) - sum(opponent_pits)

def calculate_progress(estado, jogador):
    player_score = estado.state[estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.NORTH_SCORE_PIT]
    opponent_score = estado.state[estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT]
    total_seeds = sum(estado.state[:estado.NORTH_SCORE_PIT])
    return 2 * (player_score - opponent_score) / total_seeds

def calculate_pits_difference(estado, jogador):
    player_pits = estado.state[:estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT]
    opponent_pits = estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[:estado.SOUTH_SCORE_PIT]
    
    player_pit_values = [min(pit, 6) for pit in player_pits]
    opponent_pit_values = [min(pit, 6) for pit in opponent_pits]
    
    return sum(opponent_pit_values) - sum(player_pit_values)

def calculate_opponent_capture_penalty(estado, jogador):
    next_states = [estado.real_move(move) for move in estado.get_legal_moves()]
    opponent_capture_penalty = 0
    
    for state in next_states:
        south_pieces = sum(state.state[:state.SOUTH_SCORE_PIT])
        north_pieces = sum(state.state[state.SOUTH_SCORE_PIT:state.NORTH_SCORE_PIT])
        opponent_capture_penalty += (south_pieces - north_pieces) * 0.05
    
    return opponent_capture_penalty

def calculate_empty_houses_near_store(state, jogador):
    # Determine as lojas (stores) dos jogadores
    our_store = state.SOUTH_SCORE_PIT if jogador == 0 else state.NORTH_SCORE_PIT
    opponent_store = state.NORTH_SCORE_PIT if jogador == 0 else state.SOUTH_SCORE_PIT

    # Obtém os pits relevantes para casas vazias próximas à nossa loja e à loja do oponente
    empty_houses_near_our_store = sum(1 for seeds in state.state[our_store+1:opponent_store] if seeds == 0)
    empty_houses_near_opponent_store = sum(1 for seeds in state.state[opponent_store+1:] if seeds == 0)

    return empty_houses_near_our_store - empty_houses_near_opponent_store

def func_44_teste1(estado, jogador):
    # Define weights for different evaluation factors
    SCORE_WEIGHT = 4
    SEEDS_WEIGHT = 1
    PROGRESS_WEIGHT = 2
    PITS_WEIGHT = 3
    EMPTY_HOUSES_WEIGHT = 2  # Peso para as casas vazias próximas às lojas

    # Calculate the final evaluation value
    evaluation = (
        SCORE_WEIGHT * calculate_score_difference(estado, jogador) +
        SEEDS_WEIGHT * calculate_seeds_difference(estado, jogador) +
        PROGRESS_WEIGHT * calculate_progress(estado, jogador) +
        PITS_WEIGHT * calculate_pits_difference(estado, jogador) -
        calculate_opponent_capture_penalty(estado, jogador) +
        EMPTY_HOUSES_WEIGHT * calculate_empty_houses_near_store(estado, jogador)
    )

    return evaluation

In [66]:
def calculate_score_difference(state, player):
    player_store, opponent_store = (state.SOUTH_SCORE_PIT, state.NORTH_SCORE_PIT) if player == 0 else (state.NORTH_SCORE_PIT, state.SOUTH_SCORE_PIT)
    return state.stone_count(player_store) - state.stone_count(opponent_store)

def calculate_seeds_difference(state, player):
    our_pits = state.our_pits(player)
    opponent_pits = state.opponent_pits(player)
    return sum(our_pits) - sum(opponent_pits)

def calculate_progress(state, player):
    player_store, opponent_store = (state.SOUTH_SCORE_PIT, state.NORTH_SCORE_PIT) if player == 0 else (state.NORTH_SCORE_PIT, state.SOUTH_SCORE_PIT)
    player_score = state.stone_count(player_store)
    opponent_score = state.stone_count(opponent_store)
    total_seeds = sum(state.pits)
    return 2 * (player_score - opponent_score) / total_seeds

def calculate_pits_difference(state, player):
    our_pits = state.our_pits(player)
    opponent_pits = state.opponent_pits(player)
    
    player_pit_values = [min(pit, 6) for pit in our_pits]
    opponent_pit_values = [min(pit, 6) for pit in opponent_pits]
    
    return sum(opponent_pit_values) - sum(player_pit_values)

def calculate_opponent_capture_penalty(state, player):
    legal_moves = state.legal_moves(player)
    opponent_capture_penalty = 0
    
    for move in legal_moves:
        next_state = state.result(move)
        south_pieces = next_state.stone_count(state.SOUTH_SCORE_PIT)
        north_pieces = next_state.stone_count(state.NORTH_SCORE_PIT)
        opponent_capture_penalty += (south_pieces - north_pieces) * 0.05
    
    return opponent_capture_penalty

def calculate_empty_pits(state, player):
    our_pits = state.our_pits(player)
    opponent_pits = state.opponent_pits(player)
    
    player_empty_pits = our_pits.count(0)
    opponent_empty_pits = opponent_pits.count(0)
    return player_empty_pits - opponent_empty_pits

def calculate_opponent_mobility(state, player):
    opponent = 1 - player
    legal_moves = state.legal_moves(opponent)
    return len(legal_moves)

def calculate_block(state, player):
    legal_moves = state.legal_moves(player)
    block = 0
    
    for move in legal_moves:
        next_state = state.result(move)
        opponent_legal_moves = next_state.legal_moves(1 - player)
        
        if not opponent_legal_moves:
            block += 1
        
    return block

def calculate_final_houses_control(state, player):
    our_store, opponent_store = (state.SOUTH_SCORE_PIT, state.NORTH_SCORE_PIT) if player == 0 else (state.NORTH_SCORE_PIT, state.SOUTH_SCORE_PIT)
    our_final_houses = state.pits[our_store + 1:opponent_store]
    opponent_final_houses = state.pits[opponent_store + 1:]

    our_control = our_final_houses.count(seeds for seeds in our_final_houses if seeds > 0)
    opponent_control = opponent_final_houses.count(seeds for seeds in opponent_final_houses if seeds > 0)

    return our_control - opponent_control

def calculate_capture_moves(state, player):
    legal_moves = state.legal_moves(player)
    capture_moves = 0
    
    for move in legal_moves:
        next_state = state.result(move)
        player_score = next_state.stone_count(state.SOUTH_SCORE_PIT) if player == 0 else next_state.stone_count(state.NORTH_SCORE_PIT)
        opponent_score = next_state.stone_count(state.NORTH_SCORE_PIT) if player == 0 else next_state.stone_count(state.SOUTH_SCORE_PIT)
        
        if player_score > opponent_score:
            capture_moves += 1
        
    return capture_moves

def func_44_teste2(state, player):
    SCORE_WEIGHT = 4
    SEEDS_WEIGHT = 1
    PROGRESS_WEIGHT = 2
    PITS_WEIGHT = 3
    MOBILITY_WEIGHT = 1
    BLOCK_WEIGHT = 1
    EMPTY_HOUSES_WEIGHT = 2
    STRATEGIC_WEIGHT = 1
    CONTROL_WEIGHT = 2
    CAPTURE_WEIGHT = 1

    evaluation = (
        SCORE_WEIGHT * calculate_score_difference(state, player) +
        SEEDS_WEIGHT * calculate_seeds_difference(state, player) +
        PROGRESS_WEIGHT * calculate_progress(state, player) +
        PITS_WEIGHT * calculate_pits_difference(state, player) -
        calculate_opponent_capture_penalty(state, player) +
        EMPTY_HOUSES_WEIGHT * calculate_empty_pits(state, player) +
        MOBILITY_WEIGHT * calculate_opponent_mobility(state, player) +
        BLOCK_WEIGHT * calculate_block(state, player) +
        calculate_final_houses_control(state, player) +
        CAPTURE_WEIGHT * calculate_capture_moves(state, player)
    )

    return evaluation

Teste - Jogador com pesos diferentes

In [67]:
def calculate_score_difference(estado, jogador):
    player_score = estado.state[estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.NORTH_SCORE_PIT]
    opponent_score = estado.state[estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT]
    return player_score - opponent_score

def calculate_seeds_difference(estado, jogador):
    player_pits = estado.state[:estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT]
    opponent_pits = estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[:estado.SOUTH_SCORE_PIT]
    return sum(player_pits) - sum(opponent_pits)

def calculate_progress(estado, jogador):
    player_score = estado.state[estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.NORTH_SCORE_PIT]
    opponent_score = estado.state[estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT]
    total_seeds = sum(estado.state[:estado.NORTH_SCORE_PIT])
    return 2 * (player_score - opponent_score) / total_seeds

def calculate_pits_difference(estado, jogador):
    player_pits = estado.state[:estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT]
    opponent_pits = estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[:estado.SOUTH_SCORE_PIT]
    
    player_pit_values = [min(pit, 6) for pit in player_pits]
    opponent_pit_values = [min(pit, 6) for pit in opponent_pits]
    
    return sum(opponent_pit_values) - sum(player_pit_values)

def calculate_opponent_capture_penalty(estado, jogador):
    next_states = [estado.real_move(move) for move in estado.get_legal_moves()]
    opponent_capture_penalty = 0
    
    for state in next_states:
        south_pieces = sum(state.state[:state.SOUTH_SCORE_PIT])
        north_pieces = sum(state.state[state.SOUTH_SCORE_PIT:state.NORTH_SCORE_PIT])
        opponent_capture_penalty += (south_pieces - north_pieces) * 0.05
    
    return opponent_capture_penalty

def func_44_otimo(estado, jogador):
    # Define weights for different evaluation factors
    SCORE_WEIGHT = 4
    SEEDS_WEIGHT = 1
    PROGRESS_WEIGHT = 2
    PITS_WEIGHT = 3

    # Calculate the final evaluation value
    evaluation = (
        SCORE_WEIGHT * calculate_score_difference(estado, jogador) +
        SEEDS_WEIGHT * calculate_seeds_difference(estado, jogador) +
        PROGRESS_WEIGHT * calculate_progress(estado, jogador) +
        PITS_WEIGHT * calculate_pits_difference(estado, jogador) -
        calculate_opponent_capture_penalty(estado, jogador)
    )

    return evaluation

In [68]:
def chapiteau_algorithm(estado, jogador):
    if estado.is_game_over():
        aux = estado.result()
        return aux*100 if jogador == estado.SOUTH else aux*-100
    
    player_pits = estado.state[:estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT]
    opponent_pits = estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[:estado.SOUTH_SCORE_PIT]
    return sum(player_pits) - sum(opponent_pits)

Testes de empenho

In [71]:
grupo44 = JogadorAlfaBeta('Teste', 6, func_44_teste1)
testeAUX = JogadorAlfaBeta('Fortissimo', 6, func_44_otimo)
#chapiteau = JogadorAlfaBeta('Chapiteau', 6, chapiteau_algorithm)
jogo=Kalah(10)
print(jogaNpares(jogo,100,grupo44,testeAUX))

KeyboardInterrupt: 

In [70]:

({'Teste': 40, 'Fortissimo': 56, 'Empate': 4}, {'Teste': 60, 'Fortissimo': 36, 'Empate': 4}, {'Teste': 100, 'Fortissimo': 92, 'Empate': 8}, {'Teste': 308, 'Fortissimo': 284})


({'Teste': 40, 'Fortissimo': 56, 'Empate': 4},
 {'Teste': 60, 'Fortissimo': 36, 'Empate': 4},
 {'Teste': 100, 'Fortissimo': 92, 'Empate': 8},
 {'Teste': 308, 'Fortissimo': 284})